In [1]:
import pickle
import torch
from torch import nn
import torchvision
import torch.optim as optim
from tqdm import tqdm_notebook as tqdm

In [2]:
from modules.utils import *

In [3]:
ensembl_trx = pickle.load(open('data/ensembl_trx.pkl', 'rb'))
trx_orfs = pickle.load(open('data/trx_orfs.pkl', 'rb'))
dataset = pickle.load(open('data/dataset.pkl', 'rb'))
pseudogene_dataset = pickle.load(open('data/pseudogene_dataset.pkl', 'rb'))

In [4]:
clustering_dataset = {x:y["sequence"] for (x,y) in ensembl_trx.items() if x in dataset or x in pseudogene_dataset}

In [6]:
data = [(x,y) for x,y in clustering_dataset.items()]

In [8]:
def build_fasta(data, filename):
    fasta_sequences = []
    for i, trx in enumerate(data):
        trx_id, seq = trx[0], trx[1]
        header = f'>{i} {trx_id}'
        fasta_sequence = header + '\n' + seq.upper()
        fasta_sequences.append(fasta_sequence)
    
    fasta_text = '\n'.join(fasta_sequences)
    
    with open(filename + '.fasta', 'w') as fasta_file:
        fasta_file.write(fasta_text)

In [9]:
build_fasta(data, 'mydb')

In [5]:
psicube_trx = dict()
with open('psicube_trx.txt', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    for n, row in enumerate(reader):
        if n == 0:
            cols = row
            continue
        line = dict(zip(cols, row))
        pseudo_trx = line["ID"].split(".")[0]
        if "Parent transcript" not in line:
            continue
        parent_trx = line["Parent transcript"].split(".")[0]
        psicube_trx[pseudo_trx] = parent_trx

In [6]:
len(psicube_trx)

10370

In [7]:
cluster_dict = {}
current_cluster = ''
with open('output.txt', 'r') as f:
    for line in f:
        if line.startswith('#'):
            current_cluster = line.strip()
            cluster_dict[current_cluster] = []
        else:
            _, enst = line.strip().split()
            cluster_dict[current_cluster].append(enst)

In [8]:
len(cluster_dict)

16353

In [16]:
count = 0
check = 0
for cluster, trxps in cluster_dict.items():
    for trx in trxps:
        if trx in psicube_trx and psicube_trx[trx] in dataset:
            check += 1
            parent = psicube_trx[trx]
            if parent in trxps:
                count += 1
        else:
            continue

In [17]:
count

4974

In [18]:
check

6599

In [20]:
strings = [x for x in cluster_dict.keys()]
random.seed(5)
random.shuffle(strings)
bins = np.array_split(strings, 5)
split_dict = dict()
for idx, bin_ in enumerate(bins):
    for cluster in bin_:
        for trx in cluster_dict[cluster]:
            split_dict[trx] = idx

In [21]:
from collections import Counter

In [22]:
Counter([x for x in split_dict.values()])

Counter({0: 13488, 1: 13018, 2: 12822, 3: 13678, 4: 13290})

In [66]:
pickle.dump(split_dict, open('data/split_dict.pkl', 'wb'))